In [1]:
import numpy as np
from glob import glob
from pathlib import Path
import torch
from matplotlib import pyplot as plt
from tqdm import tqdm
import pickle

In [2]:
data_path = Path('/common/users/dm1487/legged_manipulation_data/rollout_data/random_3obstacle_data_bb_weight_move_1/train')

In [3]:
all_pieces = sorted(glob(str(data_path/'*.npz')))[:-1]
print(len(all_pieces))
idxs = np.arange(500, len(all_pieces))
pieces = [all_pieces[i] for i in idxs]
len(pieces)

2458


1958

In [6]:
def split_and_pad_trajectories(tensor, dones):
    """ Splits trajectories at done indices. Then concatenates them and padds with zeros up to the length og the longest trajectory.
    Returns masks corresponding to valid parts of the trajectories
    Example: 
        Input: [ [a1, a2, a3, a4 | a5, a6],
                 [b1, b2 | b3, b4, b5 | b6]
                f]

        Output:[ [a1, a2, a3, a4], | [  [True, True, True, True],
                 [a5, a6, 0, 0],   |    [True, True, False, False],
                 [b1, b2, 0, 0],   |    [True, True, False, False],
                 [b3, b4, b5, 0],  |    [True, True, True, False],
                 [b6, 0, 0, 0]     |    [True, False, False, False],
                ]                  | ]    
            
    Assumes that the input has the following dimension order: [time, number of envs, aditional dimensions]
    """
    # dones = dones.clone()
    dones[-1] = 1
    # Permute the buffers to have order (num_envs, num_transitions_per_env, ...), for correct reshaping
    flat_dones = dones.transpose(1, 0).reshape(-1, 1)
    # flat_dones = dones.reshape(-1, 1)

    # Get length of trajectory by counting the number of successive not done elements
    done_indices = torch.cat((flat_dones.new_tensor([-1], dtype=torch.int64), flat_dones.nonzero()[:, 0]))
    trajectory_lengths = done_indices[1:] - done_indices[:-1]
    trajectory_lengths_list = trajectory_lengths.tolist()
    # Extract the individual trajectories
    trajectories = torch.split(tensor.transpose(1, 0).flatten(0, 1),trajectory_lengths_list)
    padded_trajectories = torch.nn.utils.rnn.pad_sequence(trajectories)
    
    trajectory_masks = trajectory_lengths > torch.arange(0, tensor.shape[0], device=tensor.device).unsqueeze(1)
    
    print(trajectory_masks.shape)
    return padded_trajectories, trajectory_masks

In [7]:
keys = list(np.load(pieces[0]).keys())
one_piece = np.load(pieces[0])
one_piece = np.load(pieces[0])
for key in keys:
    print(key, one_piece[key].shape)

observations (25, 4055, 13)
privileged_observations (25, 4055, 24)
observation_histories (25, 4055, 37)
full_seen_world (25, 4055, 24)
dones (25, 4055, 1)


In [8]:
def get_pieces(key):
    tensor_pieces = []
    done_pieces = []
    start = 0
    offset = 100
    with tqdm(total=len(pieces)) as pbar:
        while True:
            tensor = None
            if start >= len(pieces):
                break
            for p in pieces[start:start+offset]:
                traj_dict = np.load(p)
                if tensor is None:
                    if key == "observation_histories":
                        tensor = torch.tensor(traj_dict[key][:, :, -37:])
                    else:
                        tensor = torch.tensor(traj_dict[key])
                    dones = torch.tensor(traj_dict['dones'], dtype=torch.bool)
                    continue
                
                if key == "observation_histories":
                    tensor = torch.cat([tensor, torch.tensor(traj_dict[key])[:, :, -37:]], dim=0)
                else:
                    tensor = torch.cat([tensor, torch.tensor(traj_dict[key])], dim=0)
                dones = torch.cat([dones, torch.tensor(traj_dict['dones'], dtype=torch.bool)], dim=0)
            tensor_pieces.append(tensor)
            done_pieces.append(dones)
            start += offset
            pbar.update(offset)
            
        return tensor_pieces, done_pieces

In [9]:
import pickle

tmp_path = Path(f'/common/users/dm1487/tmp/3_random_bb_weight_move')
tmp_path.mkdir(parents=True, exist_ok=True)

In [10]:
save_dones = False
for key in keys:
    a, d = get_pieces(key)
    with open(tmp_path/f'{key}.pkl', 'wb') as f:
        pickle.dump(a, f)
    if not save_dones:
        with open(tmp_path/f'dones.pkl', 'wb') as f:
            pickle.dump(d, f)
        print('done')
        save_dones = True

2000it [08:50,  3.77it/s]                                                                                        


done


2000it [11:22,  2.93it/s]                                                                                        
2000it [21:01,  1.59it/s]                                                                                        
2000it [11:02,  3.02it/s]                                                                                        
2000it [01:22, 24.28it/s]                                                                                        


In [11]:
def convert_to_traj_and_save(name, tensor_pieces, done_pieces, ctr = 0):
    DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/trajectories_train_bb_1/{name}')
    DATA_PATH_TRAJ.mkdir(parents=True, exist_ok=True)
    all_dones = torch.cat(done_pieces, dim=0)
    all_tensor = torch.cat(tensor_pieces, dim=0)
    all_tensor_traj = split_and_pad_trajectories(all_tensor, all_dones)
    if name == 'dones':
        traj = all_tensor_traj[1].permute(1, 0)
    else:
        traj = all_tensor_traj[0].permute(1, 0, 2)
    start = 0
    offset = 10000
    with tqdm(total=traj.shape[0]) as pbar:
        while True:
            if start >= traj.shape[0]:
                break

            np.savez_compressed(DATA_PATH_TRAJ/f'{name}_{ctr}.npz', data=traj[(start+1):(start+offset-1)])
            start += offset
            pbar.update(offset)
            ctr += 1
    return all_tensor_traj[1], ctr

In [13]:
with open(tmp_path/f'dones.pkl', 'rb') as f:
    d = pickle.load(f)
    
for key in [*keys[:], 'dones']:
# for key in [*keys[:]]:
    a = None
    b = None
    
    print('loading...')
    with open(tmp_path/f'{key}.pkl', 'rb') as f:
        a = pickle.load(f)
    print('done')
    print(len(a))
    
    splits = 5
    start = 0
    offset = len(a)//splits
    ctr = 0
    for _ in range(splits):
        _, ctr = convert_to_traj_and_save(key, a[start:(start+offset)], d[start:(start+offset)], ctr)
        start += offset

loading...
done
20
torch.Size([10000, 142925])


150000it [02:18, 1082.00it/s]                                                                                    


torch.Size([10000, 154737])


160000it [05:37, 473.69it/s]                                                                                     


torch.Size([10000, 156310])


...

160000it [00:50, 3155.06it/s]                                                                                    


torch.Size([10000, 157113])


160000it [00:49, 3229.81it/s]                                                                                    


torch.Size([8950, 142591])


150000it [00:40, 3743.13it/s]                                                                                    


In [14]:
DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/traj_list_train_bb_traj_1')
DATA_PATH_TRAJ.mkdir(parents=True, exist_ok=True)

file_list = {}
for key in [*keys, 'dones']:
    file_list[key] = sorted(list(glob(f'/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/trajectories_train_bb_1/{key}/*.npz')), 
                            key= lambda x: int(str(x).split('.npz')[0].split('/')[-1].split('_')[-1]))

traj_ctr = 0
with tqdm(total=len(list(file_list.values())[0])) as pbar:
    for observations, privileged_observations, observation_histories, full_seen_world, dones in zip(*file_list.values()):
        obs = np.load(observations)['data']
        priv_obs = np.load(privileged_observations)['data']
        obs_hist = np.load(observation_histories)['data']
        fsw = np.load(full_seen_world)['data']
        d = np.load(dones)['data']
        for idx in range(obs.shape[0]):
            # print(obs_hist[idx].shape, fsw[idx].shape, priv_obs[idx].shape, d[idx][:750].reshape(-1, 1).shape)
            np.savez_compressed(DATA_PATH_TRAJ/f'traj_{traj_ctr}', obs=obs[idx], priv_obs=priv_obs[idx], obs_hist=obs_hist[idx][:, -37:], fsw=fsw[idx], done=d[idx][:750].reshape(-1, 1))
            traj_ctr += 1
        pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████| 78/78 [5:32:21<00:00, 255.66s/it]


In [4]:
from pathlib import Path
import os
import numpy as np
from tqdm import tqdm
import pickle
DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/traj_list_train_bb_traj_1')
all_files = os.listdir(DATA_PATH_TRAJ)
ignore_files = []
label_idx = {1: [], 2:[], 3:[]}
for i in tqdm(all_files[:]):
    traj = np.load(DATA_PATH_TRAJ/i)
    
    key_id = int(traj['fsw'][0][2]!=0) + int(traj['fsw'][0][9]!=0) + int(traj['fsw'][0][16]!=0)
    if key_id == 0:
        continue
    label_idx[int(key_id)].append(DATA_PATH_TRAJ/i)
    
    first_idx = 0
    if np.sum(traj['priv_obs'][first_idx][:]) > 0:
        ignore_files.append(DATA_PATH_TRAJ/i)
        continue
        
    last_idx = traj['done'].nonzero()[0][-1]
    if np.sum(traj['fsw'][last_idx][:7]) != 0:
        for j in range(3):
            k = j*7 + 1
            if np.sum(traj['fsw'][last_idx][k:k+6] == traj['priv_obs'][last_idx][k:k+6]) != 6:
                ignore_files.append(DATA_PATH_TRAJ/i)
                break
## save ignore_files as pickle

100%|████████████████████| 753525/753525 [3:04:25<00:00, 68.10it/s]


In [20]:
# with open('/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/ignore_random_traj_bb.pkl', 'wb') as f:
#     pickle.dump(ignore_files, f)


In [19]:
len(ignore_files)

202065

In [53]:
from pathlib import Path
import os
import numpy as np
from tqdm import tqdm
import pickle
DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/traj_list_train_bb_traj')
all_files = os.listdir(DATA_PATH_TRAJ)
label_idx = {1: [], 2:[], 3:[]}
# la = []
ctr = 0
for i in tqdm(all_files[:]):
    traj = np.load(DATA_PATH_TRAJ/i)
    key_id = int(traj['fsw'][0][2]!=0) + int(traj['fsw'][0][9]!=0) + int(traj['fsw'][0][16]!=0)
    label_idx[int(key_id)].append(DATA_PATH_TRAJ/i)

100%|█████████████████████████████████████████████████████████| 736704/736704 [1:29:19<00:00, 137.46it/s]


In [54]:
# with open('/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/labels_random_traj_bb.pkl', 'wb') as f:
#     pickle.dump(label_idx, f)

In [58]:
ignore_files = []
with open('/common/users/dm1487/legged_manipulation_data/rollout_data/3_obstacle/ignore_random_traj_bb.pkl', 'rb') as f:
    ignore_files = pickle.load(f)

In [86]:
# list(set(label_idx[3]) - set(ignore_files))
idxs_valid = {1: [], 2:[], 3:[]}
for k, v in label_idx.items():
    idxs_valid[k] = [int(i.stem.split('_')[-1]) for i in list(set(v) - set(ignore_files))]
limit_class = min(len(idxs_valid[1][:-2000]), len(idxs_valid[2][:-2000]), len(idxs_valid[3][:-2000]))
train_idxs_valid_all = []
for k, v in idxs_valid.items():
    train_idxs_valid_all.extend(np.random.choice(v[:-2000], limit_class))
val_idxs_valid_all = []
for k, v in idxs_valid.items():
    val_idxs_valid_all.extend(np.random.choice(v[-2000:], 2000))

In [87]:
len(train_idxs_valid_all), len(val_idxs_valid_all)

(370854, 6000)

In [ ]:
import numpy as np
from glob import glob
from pathlib import Path
import torch
from matplotlib import pyplot as plt

In [ ]:
data_path = Path('/common/users/dm1487/legged_manipulation_data/rollout_data/set_3_trajectories')

In [ ]:
files_merge = sorted(list((data_path/'obs_hist').glob('*')), key= lambda x: int(str(x).split('.npz')[0].split('_')[-1]))
DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/set_3_trajectories/obs_hist_combined')
DATA_PATH_TRAJ.mkdir(parents=True, exist_ok=True)
# len(files_merge)
for i in range(10, len(files_merge), 10):
    files_length = 10
    if i == 50:
        files_length = 9
    files = [np.load(files_merge[j+i]) for j in range(files_length)] # np.load(files_merge[i]), np.load(files_merge[i+1])
    print(i, len(files))
    final_data = np.concatenate([f['data'] for f in files], axis=0)
    print(final_data.shape)
    np.savez_compressed(DATA_PATH_TRAJ/f'obs_hist_combined_{i}.npz', data=final_data)
    # break

In [ ]:
trajectories = 200000//100000
traj_data = {
    'obs': None, 
    'obs_hist_combined': None, 
    'priv_obs': None, 
    'mask': None
}
# while length <= trajectory_length:
for i in sorted(data_path.glob('*')):
    name = str(i).split('/')[-1] 
    if  name == 'obs_hist' or name == 'mask1':
        continue
    print(i)
    for j in sorted(glob(f'{i}/*.npz'))[:trajectories]:
        if traj_data[name] is None:
            traj_data[name] = np.load(j)['data']
        else:
            traj_data[name] = np.concatenate([traj_data[name], np.load(j)['data']], axis=0)
        print(traj_data[name].shape)

In [ ]:
traj_data['obs_hist_combined'].shape

In [ ]:
traj_path = Path('/common/users/dm1487/legged_manipulation_data/rollout_data/set3_traj_200k')
# traj_path.mkdir(exist_ok=True, parents=True)

In [ ]:
# np.savez_compressed(traj_path, **traj_data)

In [ ]:
# traj_num = np.random.randint(0, 200000)
# print(traj_num)
# plt.scatter(traj_data['obs'][traj_num][:, 0], traj_data['obs'][traj_num][:, 1])

In [ ]:
trajectories = 200000
obs_hist = torch.zeros(trajectories, traj_data['obs'].shape[1],  traj_data['obs'].shape[2] + traj_data['obs_hist_combined'].shape[2])
obs_hist[:, :, :traj_data['obs'].shape[2]] = torch.tensor(traj_data['obs'])
obs_hist[:, :, traj_data['obs'].shape[2]:] = torch.tensor(traj_data['obs_hist_combined'])

FILE_NAME = 'rnn_200k_data'
DATA_PATH_TRAJ = Path(f'/common/users/dm1487/legged_manipulation_data/rollout_data/set_3_trajectories/{FILE_NAME}')
np.savez_compressed(DATA_PATH_TRAJ, inp=obs_hist.numpy(), target=traj_data['priv_obs'], mask=traj_data['mask'])

In [ ]:
print(DATA_PATH_TRAJ)
traj = np.load(DATA_PATH_TRAJ/'traj_3.npz')['obs']
plt.scatter(traj[:, 0], traj[:, 1])

In [ ]:
traj.shape